In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random

### Find new technology fields appearing after 1990

In [2]:
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas()
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]
# Add MSA subfield
pdf["msa_pc"] = list(zip(pdf["inv_msa"], pdf["primclass"]))

In [3]:
# Get number of primary class patents by MSA, and year of their first appearance
p2 = pdf.groupby(["inv_msa", "primclass"]).size()
# Only get primary classes with more than 50 patents
p2 = p2.loc[p2 >= 50].reset_index().rename(columns={0:"size"})
# Date of first appearance
p3 = pdf[["inv_msa", "primclass", "gyear"]].sort_values(["inv_msa", "primclass", "gyear"]).drop_duplicates(["inv_msa", "primclass"])
# Merge
p2 = p2.merge(p3, how="left", on=["inv_msa", "primclass"]).rename(columns={"gyear": "first_gyear"})

# New technology fields after 1990
p3 = p2.loc[p2["first_gyear"] >= 1990]

# Subfield
p3["msa_pc"] = list(zip(p3["inv_msa"], p3["primclass"]))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
# tp: patent from new subfield
# Primary class random sample
ns = fastparquet.ParquetFile("DataStore/2018-07-P2/Reg0726/reg_naics_name_sim_tr_0726.parq").to_pandas()
ns["op_msa_pc"] = list(zip(ns["op_inv_msa"], ns["op_primclass"]))
ns["tp_msa_pc"] = list(zip(ns["tp_inv_msa"], ns["tp_primclass"]))
ns = ns.loc[ns["tp_msa_pc"].isin(p3["msa_pc"])]

In [5]:
# Find highly similar PC
ns2 = ns.loc[(ns["norm_sim_mean_docvecs_pc"] >= 1) &
             (ns["primclass_match"] == False), ["tp_msa_pc", "op_msa_pc",
"tp_primclass", "op_primclass", "tp_inv_msa", "op_inv_msa", "inv_msa_match", "year_group", 
"sim_mean_docvecs_pc", "norm_sim_mean_docvecs_pc"]].drop_duplicates()
ns2["tp_control_msa_pc"] = list(zip(ns2["tp_inv_msa"], ns2["op_primclass"]))

# Comparison groups
ns3 = ns2[["tp_msa_pc", "op_msa_pc", "tp_control_msa_pc"]]

ns3["tp_first_gyear"] = ns3["tp_msa_pc"].map(dict(zip(p3["msa_pc"], p3["first_gyear"])))

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [6]:
ns2

,tp_msa_pc,op_msa_pc,tp_primclass,op_primclass,tp_inv_msa,op_inv_msa,inv_msa_match,year_group,sim_mean_docvecs_pc,norm_sim_mean_docvecs_pc,tp_control_msa_pc
index,,,,,,,,,,,
3199,"(Rochester, MN, 726.0)","(Seattle-Tacoma-Bellevue, WA, 713.0)",726.0,713.0,"Rochester, MN","Seattle-Tacoma-Bellevue, WA",False,2005-15,0.882249,1.098024,"(Rochester, MN, 713.0)"
4428,"(Boston-Cambridge-Quincy, MA-NH, 719.0)","(Boston-Cambridge-Quincy, MA-NH, 709.0)",719.0,709.0,"Boston-Cambridge-Quincy, MA-NH","Boston-Cambridge-Quincy, MA-NH",True,2005-15,0.882839,1.100979,"(Boston-Cambridge-Quincy, MA-NH, 709.0)"
7924,"(San Diego-Carlsbad-San Marcos, CA, 726.0)","(San Francisco-Oakland-Fremont, CA, 709.0)",726.0,709.0,"San Diego-Carlsbad-San Marcos, CA","San Francisco-Oakland-Fremont, CA",False,2005-15,0.865312,1.013186,"(San Diego-Carlsbad-San Marcos, CA, 709.0)"
8231,"(New York-Northern New Jersey-Long Island, NY-...","(San Jose-Sunnyvale-Santa Clara, CA, 709.0)",719.0,709.0,"New York-Northern New Jersey-Long Island, NY-N...","San Jose-Sunnyvale-Santa Clara, CA",False,2005-15,0.895246,1.163123,"(New York-Northern New Jersey-Long Island, NY-..."
8283,"(Orlando-Kissimmee-Sanford, FL, 257.0)","(Poughkeepsie-Newburgh-Middletown, NY, 438.0)",257.0,438.0,"Orlando-Kissimmee-Sanford, FL","Poughkeepsie-Newburgh-Middletown, NY",False,1995-05,0.935660,1.365552,"(Orlando-Kissimmee-Sanford, FL, 438.0)"
12306,"(Seattle-Tacoma-Bellevue, WA, 726.0)","(San Francisco-Oakland-Fremont, CA, 709.0)",726.0,709.0,"Seattle-Tacoma-Bellevue, WA","San Francisco-Oakland-Fremont, CA",False,2005-15,0.865312,1.013186,"(Seattle-Tacoma-Bellevue, WA, 709.0)"
19954,"(Orlando-Kissimmee-Sanford, FL, 257.0)","(Orlando-Kissimmee-Sanford, FL, 438.0)",257.0,438.0,"Orlando-Kissimmee-Sanford, FL","Orlando-Kissimmee-Sanford, FL",True,1995-05,0.935660,1.365552,"(Orlando-Kissimmee-Sanford, FL, 438.0)"
19985,"(Atlanta-Sandy Springs-Marietta, GA, 709.0)","(Tucson, AZ, 719.0)",709.0,719.0,"Atlanta-Sandy Springs-Marietta, GA","Tucson, AZ",False,2005-15,0.866872,1.021000,"(Atlanta-Sandy Springs-Marietta, GA, 719.0)"
21993,"(San Francisco-Oakland-Fremont, CA, 707.0)","(San Francisco-Oakland-Fremont, CA, 715.0)",707.0,715.0,"San Francisco-Oakland-Fremont, CA","San Francisco-Oakland-Fremont, CA",True,2005-15,0.869045,1.031883,"(San Francisco-Oakland-Fremont, CA, 715.0)"


In [60]:
%%time
# Use only patents from relevant MSA-PC
p2 = pdf.loc[pdf["msa_pc"].isin(ns3["tp_msa_pc"]) | pdf["msa_pc"].isin(ns3["op_msa_pc"]) | pdf["msa_pc"].isin(ns3["tp_control_msa_pc"])]

# Patents by year
%time pats_by_yr = {n:g["patent"].tolist() for n,g in p2[["msa_pc", "gyear", "patent"]].groupby(["msa_pc", "gyear"])}

# Patents 5 years after
pats_by_5_yr = {}
for yr in range(1976,2010):
    p = p2.loc[p2["gyear"].isin(range(yr,yr+5))]
    d = {(n,yr): g["patent"].tolist() for n,g in p[["msa_pc", "patent"]].groupby(["msa_pc"])}
    pats_by_5_yr.update(d)

CPU times: user 2.82 s, sys: 22.7 ms, total: 2.84 s
Wall time: 2.72 s
CPU times: user 7.1 s, sys: 85.3 ms, total: 7.19 s
Wall time: 6.86 s


In [67]:
# Add a grant year for each of the 10 years following the first grant year of new subfield
ns4 = [list(zip([r1]*10, [r2]*10, [r3]*10, list(range(r3, r3+10)), [r4]*10)) for r1,r2,r3,r4\
       in zip(ns3["tp_msa_pc"], ns3["op_msa_pc"], ns3["tp_first_gyear"], ns3["tp_control_msa_pc"])]
ns4 = [item for sublist in ns4 for item in sublist]
ns4 = pd.DataFrame({"tp_msa_pc": [i[0] for i in ns4],
                   "op_msa_pc": [i[1] for i in ns4],
                    "tp_control_msa_pc": [i[4] for i in ns4],
                   "tp_first_gyear": [i[2] for i in ns4],
                   "gyear": [i[3] for i in ns4],})

In [72]:
# Patents for each MSA-PC
ns4["tp_msa_pc_pats"] = [pats_by_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["tp_msa_pc"], ns4["gyear"])]
ns4["op_msa_pc_pats"] = [pats_by_5_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["op_msa_pc"], ns4["gyear"])]
ns4["tp_control_msa_pc_pats"] = [pats_by_5_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["tp_control_msa_pc"], ns4["gyear"])]
print(len(ns4))
# Drop whereever any of the patent lists are missing
ns4 = ns4.dropna(subset=['tp_msa_pc_pats', 'op_msa_pc_pats',
       'tp_control_msa_pc_pats'],how="any")
print(len(ns4))

7630
5723


In [90]:
ns4.to_pickle("DataStore/2018-08/related_field_pats_0917.pkl")
ns4.head()

,gyear,op_msa_pc,tp_control_msa_pc,tp_first_gyear,tp_msa_pc,tp_msa_pc_pats,op_msa_pc_pats,tp_control_msa_pc_pats
0,1996,"(Seattle-Tacoma-Bellevue, WA, 713.0)","(Rochester, MN, 713.0)",1996,"(Rochester, MN, 726.0)",[5506961],"[5483596, 5497492, 5526523, 5537583, 5559884, ...","[5646509, 5649177, 5657482, 5666522, 5692165, ..."
2,1998,"(Seattle-Tacoma-Bellevue, WA, 713.0)","(Rochester, MN, 713.0)",1996,"(Rochester, MN, 726.0)","[5734819, 5758068, 5832212]","[5710910, 5719941, 5748895, 5757920, 5771381, ...","[5745570, 5748957, 5758137, 5787292, 5790838, ..."
3,1999,"(Seattle-Tacoma-Bellevue, WA, 713.0)","(Rochester, MN, 713.0)",1996,"(Rochester, MN, 726.0)","[5892905, 5896499, 5908469]","[5913217, 5933630, 5948105, 5968178, 6014744, ...","[5875248, 5875327, 5911063, 5951680, 6044461, ..."
4,2000,"(Seattle-Tacoma-Bellevue, WA, 713.0)","(Rochester, MN, 713.0)",1996,"(Rochester, MN, 726.0)",[6101607],"[6014744, 6044155, 6061792, 6092208, 6108784, ...","[6044461, 6128746, 6182237, 6216226, 6351819, ..."
5,2001,"(Seattle-Tacoma-Bellevue, WA, 713.0)","(Rochester, MN, 713.0)",1996,"(Rochester, MN, 726.0)",[6330676],"[6189100, 6195750, 6209088, 6209089, 6209093, ...","[6182237, 6216226, 6351819, 6430692, 6438703, ..."


### Similarities across similar technology fields

In [2]:
ns4 = pd.read_pickle("DataStore/2018-08/related_field_pats_0917.pkl")

In [11]:
msa_pc_pats[0][0]+msa_pc_pats[0][1]

(5506961,
 5841873,
 ('Rochester, MN', 726.0),
 ('Seattle-Tacoma-Bellevue, WA', 713.0),
 1996)

In [20]:
# Get sample pairs for each
rs = {}
msa_pc_pats = [list(itertools.product(i,j)) for i,j in zip(ns4["tp_msa_pc_pats"], ns4["op_msa_pc_pats"])]
# Add identifiers
msa_pc_pats = [list(zip(i, [(j,k,l,m)]*len(i))) for i,j,k,l,m in 
                    zip(msa_pc_pats, ns4["tp_msa_pc"], ns4["op_msa_pc"], ns4["gyear"], ns4["tp_first_gyear"])]
# Sample half
msa_pc_pats = (random.sample(l, int(np.round(len(l)/2))) for l in msa_pc_pats)
msa_pc_pats = [item[0]+item[1] for sublist in msa_pc_pats for item in sublist]


rs["msa_pc_pats"] = pd.DataFrame({"tp": [i[0] for i in msa_pc_pats],
                           "op": [i[1] for i in msa_pc_pats],
                            "tp_msa_pc": [i[2] for i in msa_pc_pats],
                            "op_msa_pc": [i[3] for i in msa_pc_pats],
                            "gyear": [i[4] for i in msa_pc_pats],
                            "tp_first_gyear": [i[5] for i in msa_pc_pats],})
rs["msa_pc_pats"] = rs["msa_pc_pats"].drop_duplicates()
print(len(rs["msa_pc_pats"]))

2004752


In [22]:
# Get sample pairs for each
msa_pc_pats = [list(itertools.product(i,j)) for i,j in zip(ns4["tp_msa_pc_pats"], ns4["tp_control_msa_pc_pats"])]
# Add identifiers
msa_pc_pats = [list(zip(i, [(j,k,l,m)]*len(i))) for i,j,k,l,m in 
                    zip(msa_pc_pats, ns4["tp_msa_pc"], ns4["tp_control_msa_pc"], ns4["gyear"], ns4["tp_first_gyear"])]
# Sample half
msa_pc_pats = (random.sample(l, int(np.round(len(l)/2))) for l in msa_pc_pats)
msa_pc_pats = [item[0]+item[1] for sublist in msa_pc_pats for item in sublist]

rs["msa_control_pc_pats"] = pd.DataFrame({"tp": [i[0] for i in msa_pc_pats],
                           "op": [i[1] for i in msa_pc_pats],
                            "tp_msa_pc": [i[2] for i in msa_pc_pats],
                            "tp_control_msa_pc": [i[3] for i in msa_pc_pats],
                            "gyear": [i[4] for i in msa_pc_pats],
                            "tp_first_gyear": [i[5] for i in msa_pc_pats],})
rs["msa_control_pc_pats"] = rs["msa_control_pc_pats"].drop_duplicates()
print(len(rs["msa_control_pc_pats"] ))

246137


In [24]:
# Get similarity
def grouper(n, iterable):
    """
    >>> list(grouper(3, 'ABCDEFG'))
    [['A', 'B', 'C'], ['D', 'E', 'F'], ['G']]
    """
    iterable = iter(iterable)
    return iter(lambda: list(itertools.islice(iterable, n)), [])


import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]

print("Getting row values")
print(datetime.datetime.now())
pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
pat_dict = dict(zip(pat_dict, range(len(pat_dict))))

for dm in dms:
    print((dm,"started"))
    print("Loading matrix and dict")
    print(datetime.datetime.now())
    
    for k,v in rs.items():
    
        l2 = v.drop_duplicates()
        print(len(l2))
        # Store copy as array
        l3 = l2.loc[l2["tp"].isin(pat_dict.keys()) & l2["op"].isin(pat_dict.keys()), ["tp", "op"]].copy().drop_duplicates()

        if dm == "ldavecs":
            ncols = 60
        else:
            ncols = 100

        pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
    .to_pandas().values

        # Convert to chunks
        print("Getting chunks")
        print(datetime.datetime.now())
        # Split into chunks
        n_rows = 3000
        n_chunks = int(np.round(len(l3)/n_rows))
        tp_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["tp"].iteritems()]])
        op_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["op"].iteritems()]])
        del(pm)
        chunks = itertools.zip_longest(tp_chunks, op_chunks)

        print("Getting patent pair cosine similarity")
        print(datetime.datetime.now())
        # Cosine

        cos_dis = np.empty(len(l3))

        for r, c in enumerate(chunks):
            cos_dis[r*n_rows:r*n_rows+n_rows] = np.diag(distance.cdist(c[0],c[1], metric = "cosine"))

        l3["sim_{0}".format(dm)] = 1-cos_dis

        # Rename columns
        l2 = l2.merge(l3.drop_duplicates(), how = "left", on = ["tp", "op"])
        del(l3)
        
        # Update results
        rs[k] = l2
        print(len(l2))
        print("finished")
        print(datetime.datetime.now())
    

Getting row values
2018-09-17 18:54:10.155261
('ldavecs', 'started')
Loading matrix and dict
2018-09-17 18:54:11.789675
2004752
Getting chunks
2018-09-17 18:54:24.599590
Getting patent pair cosine similarity
2018-09-17 18:54:35.775888
2004752
finished
2018-09-17 18:58:33.885515
246137
Getting chunks
2018-09-17 18:58:43.066655
Getting patent pair cosine similarity
2018-09-17 18:58:44.295558
246137
finished
2018-09-17 18:59:13.945807
('docvecs', 'started')
Loading matrix and dict
2018-09-17 18:59:13.945922
2004752
Getting chunks
2018-09-17 18:59:36.712522
Getting patent pair cosine similarity
2018-09-17 18:59:59.495309
2004752
finished
2018-09-17 19:05:42.652654
246137
Getting chunks
2018-09-17 19:06:01.884433
Getting patent pair cosine similarity
2018-09-17 19:06:10.640234
246137
finished
2018-09-17 19:06:53.049446


In [40]:
pickle.dump(rs, open("DataStore/2018-08/related_pair_sim_0917.pkl", "wb"))

In [35]:
r1 = rs["msa_pc_pats"][["tp_msa_pc", "op_msa_pc", "gyear", "sim_docvecs"]]\
.groupby(["tp_msa_pc", "op_msa_pc", "gyear"]).mean().reset_index()\
.rename(columns={"sim_docvecs": "msa_pc_docvecs","sim_ldavecs": "msa_pc_ldavecs"})
r2 = rs["msa_control_pc_pats"][["tp_msa_pc", "tp_control_msa_pc", "gyear", "sim_docvecs"]]\
.groupby(["tp_msa_pc", "tp_control_msa_pc", "gyear"]).mean().reset_index()\
.rename(columns={"sim_docvecs": "control_msa_pc_docvecs","sim_ldavecs": "control_msa_pc_ldavecs"})

In [43]:
ns4 = ns4.merge(r1, how="left", on = ["tp_msa_pc", "op_msa_pc", "gyear"])
ns4 = ns4.merge(r2, how="left", on = ["tp_msa_pc", "tp_control_msa_pc", "gyear"])

In [46]:
ns4.to_pickle("DataStore/2018-08/related_field_pats_0917.pkl")

In [44]:
ns4["year_from_first"] = ns4["gyear"]-ns4["tp_first_gyear"]

In [45]:
ns4[["year_from_first", "control_msa_pc_docvecs", "msa_pc_docvecs"]].groupby("year_from_first").mean()

,control_msa_pc_docvecs,msa_pc_docvecs
year_from_first,,
0,0.160611,0.151429
1,0.173511,0.155046
2,0.146700,0.150711
3,0.150961,0.145775
4,0.145120,0.139823
5,0.165035,0.153659
6,0.173590,0.163713
7,0.170151,0.162103
8,0.174336,0.165968
